In [22]:
from Bio.SeqIO.FastaIO import SimpleFastaParser
import pandas as pd

In [94]:
root_directory='./data/'
fasta_filenames ={
'test': 'ec_vs_NOec_pide20_c50_test.fasta',
'train': 'ec_vs_NOec_pide20_c50_train.fasta',
'val': 'ec_vs_NOec_pide20_c50_val.fasta'    
}


identifiers = []
sequences = []

for dataset in ['train','test','val']:
    with open(root_directory + fasta_filenames[dataset]) as fasta_file:  # Will close handle cleanly

        for title, sequence in SimpleFastaParser(fasta_file):
            identifiers.append(title.split(None, 1)[0])  # First word is ID
            sequences.append(sequence)
            
sequences = pd.DataFrame(data=sequences,index=identifiers,columns=["sequences"])

In [95]:
annotations =  pd.read_csv('./data/merged_anno.txt',sep="\\t",names=['index','EC'],index_col=0)
annotations

/home/giri/Packages/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


,EC
index,
Q827Q8,6.3.5.5
Q8G830,6.5.1.2
B9DIF1,5.4.99.62
Q8CWW2,6.1.1.21
A1W892,2.7.1.130
A9V3C0,3.7.1.3
A0A3G9HHK2,2.3.1.-
Q8Y5W6,2.7.4.25
B7N556,6.1.1.3


In [103]:
combined = sequences.merge(annotations,how="left",left_index=True, right_index=True)

In [104]:
combined

,sequences,EC
Q6FIN4,MSSVQNKVETLVELFPDWKREDLLELVQEEKDTELEIIVEKITTGK...,NaN
Q2V4F4,MAVSKTTMLIVLVAIILSCVSISNARQMQRPQNVECVGGECQPKHP...,NaN
P52609,MVRTDAFFTENNINLVKKNPLHFDVNLFSSKSNAKDNDIKTFKDVL...,NaN
P34301,MIVYSVFIFAVLAISSVTGIFLPGGGGKKCGGHGEYGSGVIIGAERPKK,NaN
P40214,MEENKLSGNKPIQLATWSNQMGSPENNGNNANNGSDVQNVIQKALG...,NaN
Q9W4I5,MALHFSAQQFEGRFQSKRLNNWEYPRFSPPRPRGLQKNAKVVAANN...,NaN
P07712,MKTAVLLVCLAYVMAAILSLCASAPSWRPQGRFGKRTIPDRLPQTE...,NaN
Q9P7B8,MAEGDFKNDSTGCIGDSVEFTTFKKRTGKSLGNRRKKRGLSGWEPY...,NaN
Q8KDE1,MNKASRQKKIRELIENHEVSGQQELLGMLEKEGIVVAQATLSRDFA...,NaN
O69622,MVARHRAQAAADLASLAAAARLPSGLAAACARATLVARAMRVEHAQ...,NaN
